In [16]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [17]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [18]:
DATA_DIR = 'dataset'
assert os.path.exists(DATA_DIR), f"Data directory '{DATA_DIR}' does not exist"


In [19]:
# Parameters
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 5
NUM_CLASSES = 5

# Data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)


In [20]:
train_generator = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

Found 19717 images belonging to 5 classes.


In [21]:
validation_generator = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Found 4928 images belonging to 5 classes.


In [22]:
# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

C:\Users\jochr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [23]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
checkpoint = ModelCheckpoint('hand_gesture_model_acc.keras', monitor='val_accuracy', save_best_only=True, verbose=1)


In [25]:
# # Check if generators are yielding correctly
# batch_x, batch_y = next(train_generator)
# print("Train generator batch shape:", batch_x.shape, batch_y.shape)
# batch_x, batch_y = next(validation_generator)
# print("Validation generator batch shape:", batch_x.shape, batch_y.shape)

In [26]:
# # Train the model
# history = model.fit(
#     train_generator,
#     steps_per_epoch=train_generator.samples // BATCH_SIZE,
#     epochs=EPOCHS,
#     validation_data=validation_generator,
#     validation_steps=validation_generator.samples // BATCH_SIZE
# )

In [27]:
# Check if generators are yielding correctly
batch_x, batch_y = next(train_generator)
print("Train generator batch shape:", batch_x.shape, batch_y.shape)
batch_x, batch_y = next(validation_generator)
print("Validation generator batch shape:", batch_x.shape, batch_y.shape)

Train generator batch shape: (32, 224, 224, 3) (32, 5)
Validation generator batch shape: (32, 224, 224, 3) (32, 5)


In [28]:
# Train the model
try:
    history = model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=validation_generator,
        callbacks=[checkpoint],
        verbose=1
    )
except Exception as e:
    print(f"Error during training: {e}")

# Load the best model
model.load_weights('hand_gesture_model_acc.keras')

Epoch 1/5


C:\Users\jochr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


617/617 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3730 - loss: 1.8447
Epoch 1: val_accuracy improved from -inf to 0.29972, saving model to hand_gesture_model_acc.keras
617/617 ━━━━━━━━━━━━━━━━━━━━ 802s 1s/step - accuracy: 0.3731 - loss: 1.8442 - val_accuracy: 0.2997 - val_loss: 1.7453
Epoch 2/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5627 - loss: 1.0791
Epoch 2: val_accuracy improved from 0.29972 to 0.31169, saving model to hand_gesture_model_acc.keras
617/617 ━━━━━━━━━━━━━━━━━━━━ 920s 1s/step - accuracy: 0.5628 - loss: 1.0790 - val_accuracy: 0.3117 - val_loss: 1.7507
Epoch 3/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6742 - loss: 0.8132
Epoch 3: val_accuracy improved from 0.31169 to 0.33543, saving model to hand_gesture_model_acc.keras
617/617 ━━━━━━━━━━━━━━━━━━━━ 902s 1s/step - accuracy: 0.6742 - loss: 0.8131 - val_accuracy: 0.3354 - val_loss: 1.8297
Epoch 4/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7542 - loss: 0.6282
Epoch 4: val_accurac

In [29]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation accuracy: {accuracy * 100:.2f}%')

154/154 ━━━━━━━━━━━━━━━━━━━━ 58s 376ms/step - accuracy: 0.3484 - loss: 2.2023
Validation accuracy: 34.76%


In [31]:

# model = tf.keras.models.load_model('hand_gesture_model.keras')
# model.save('hand_gesture_model_acc.h5')
